In [1]:
import mlflow
import pandas as pd 
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

d:\Game\Mlops_mini\Python_Projects\mlops-mini-project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
# data preprocessing

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()

    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " " .join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    Text = [i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def removing_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):
    text = text.split()

    text = [y.lower() for y in text]

    return " " .join(text)

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ',text)
    text = text.replace(':',"")

    # remove extra whitespaces
    text = re.sub('\s+' , ' ',text).strip()
    return text

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

# def remove_small_sentences(df):
#     for i in range(len(df)):
#         if len(df.text.iloc[i].split()) < 3:
#             df.text.iloc[i] = np.nan

def normalize_text(df):
    try:

        df.content = df.content.apply(lambda content : lower_case(content))
        df.content = df.content.apply(lambda content : remove_stop_words(content))
        df.content = df.content.apply(lambda content : removing_numbers(content))
        df.content = df.content.apply(lambda content : removing_punctuations(content))
        df.content = df.content.apply(lambda content : removing_urls(content))
        df.content = df.content.apply(lambda content : lemmatization(content))
        return df
    except Exception as e:
        print(f'Error during text normalization {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [5]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['happiness' , 'sadness'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0 , 'happiness':1})
df.head()

C:\Users\abhay\AppData\Local\Temp\ipykernel_19936\389027361.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0 , 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [ ]:
vectorizer = CountVectorizer(max_features=1000)  # bag of words
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']


In [9]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [11]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/Abhay182005dat/Python_Projects.mlflow')
dagshub.init(repo_owner='Abhay182005dat', repo_name='Python_Projects', mlflow=True)

mlflow.set_experiment('Logistic Regression Baseline')

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\Game\Mlops_mini\Python_Projects\mlops-mini-project\venv\lib\site-packages\rich\live.py:260: UserWarning: install
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d1f38ba7-dd19-464d-be10-03f4bee70ce8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=fad2e887cb08b3db2e84421ebc57ce673ee679aff141e27791b1f6027b39f307




Accessing as Abhay182005dat

Initialized MLflow to track repo "Abhay182005dat/Python_Projects"

Repository Abhay182005dat/Python_Projects initialized!

2026/02/15 17:07:24 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/aada5d39c7174309af60094720e3edce', creation_time=1771155444196, experiment_id='0', last_update_time=1771155444196, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
with mlflow.start_run():
    # log preprocessing parameters
    mlflow.log_param("vectorizer" , "bag of Words")
    mlflow.log_param("num_features" , 1000)
    mlflow.log_param("test_size" , 0.2)

    # Model building and training 
    model = LogisticRegression()
    model.fit(X_train , y_train)

    # log model parameters 
    mlflow.log_param("model" , "Logistic Regression")

    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test , y_pred)
    precision = precision_score(y_test , y_pred)
    recall = recall_score(y_test , y_pred)
    f1 = f1_score(y_test , y_pred)

    # log evaluation metrics
    mlflow.log_metric('accuracy' , accuracy)
    mlflow.log_metric('precision' , accuracy)
    mlflow.log_metric('recall' , accuracy)
    mlflow.log_metric('f1 score' , accuracy)

    # log model
    mlflow.sklearn.log_model(model , 'model')

    # save and log the notebook
    import os 
    notebook_path  = 'exp1_baseline_model.ipynb'
    os.system(f'jupyter nbconvert --to notebook --execute --inplace {notebook_path}')
    mlflow.log_artifact(notebook_path)

    # print the results for verification
    print(f'Accuracy : {accuracy}')
    print(f'Precision : {precision}')
    print(f'Recall : {recall}')
    print(f'F1 score : {f1}')


2026/02/15 17:17:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
d:\Game\Mlops_mini\Python_Projects\mlops-mini-project\venv\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Accuracy : 0.776867469879518
Precision : 0.7690058479532164
Recall : 0.7773399014778325
F1 score : 0.7731504164625184
🏃 View run orderly-snake-508 at: https://dagshub.com/Abhay182005dat/Python_Projects.mlflow/#/experiments/0/runs/fe15a507661243c5affc91d40080e154
🧪 View experiment at: https://dagshub.com/Abhay182005dat/Python_Projects.mlflow/#/experiments/0
